In [1]:
!git clone https://github.com/meti-94/CodeOceanBERTQA.git -q
%cd CodeOceanBERTQA/code/

/content/CodeOceanBERTQA/code


In [18]:
!pip install transformers -q
!pip uninstall -y torchtext -q
!pip install torchtext==0.2.3 -q
!pip install pytorch-crf==0.7.2 -q
!pip install fuzzywuzzy -q
!pip install pyterrier -q
!pip install pattern -q
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
import os
from data import *

In [ ]:
# check if Bertified data exists
if not os.path.isfile('../data/Bertified/entities.npy'):
    reverb_lines = read_reverb('../data/Reverb1.1/reverb_wikipedia_tuples-1.1.txt')
    questions = pd.read_excel('../data/ReverbSQA/Final_Sheet_990824.xlsx', sheet_name=1, engine='openpyxl')
    index = get_tuple_frequency(reverb_lines, questions)
    index[index['Frequency']<10].to_excel('../data/ProcessedQuestions/normalized_questions.xlsx')
    combine_with_reverb(questions_path='../data/ProcessedQuestions/normalized_questions.xlsx', 
                    reverb_path='../data/Reverb1.1/reverb_wikipedia_tuples-1.1.txt')
    create_bertified_dataset()


In [ ]:
# table 1 output 
pd.read_excel('../data/Intermediate/train.xlsx').sample(6)[['triple', 'Question']]

triple  \
8639  ('Charlene Choi', 'was educated at', 'a number...   
4206              ('Quinn', 'left the show in', '2000')   
9211                   ('Clowers', 'died in', 'Sweeny')   
2217   ('Donaldson', 'was born in', 'Poulton-le-Fylde')   
6685    ('Kucinich', 'campaigned heavily in', 'Oregon')   
3247      ('Carthage', 'is a city in', 'Panola County')   

                                          Question  
8639  who was educated at a number of institutions  
4206               When did Quinn leave the show    
9211                          where Clowers died    
2217             who  was born in Poulton-le-Fylde  
6685              who campaigned heavily in Oregon  
3247                 What city is in Panola County

In [ ]:
# table 2 output
df = pd.read_csv(r'../data/Reverb1.1/reverb_wikipedia_tuples-1.1.txt', sep='\t', header=None)
reverb_columns_name = ['ExID', 'arg1', 'rel', 'arg2', 'narg1', 'nrel', 'narg2', 'csents', 'conf', 'urls']
df.columns = reverb_columns_name
df = df.dropna()
df = df.drop_duplicates()

print(f'#Triples\t:\t{len(df)}')
print(f'#Relations\t:\t{len(df["rel"].unique())}')
print(f'#Entity 1\t:\t{len(df["arg1"].unique())}')
print(f'#Entity 2\t:\t{len(df["arg2"].unique())}')
print(f'Total Unique Entities\t:\t{len(set(df["arg1"].unique().tolist()+df["arg2"].unique().tolist()))}')
vocab = df["arg1"].unique().tolist()+df["arg2"].unique().tolist()+df["rel"].unique().tolist()
vocab = list(map(lambda x:x.split(), vocab))
vocab = [item for sublist in vocab for item in sublist]
print(f'Vocabulary Size\t:\t{len(set(vocab))}')

#Triples	:	407236
#Relations	:	101977
#Entity 1	:	182329
#Entity 2	:	156166
Total Unique Entities	:	304853
Vocabulary Size	:	148953


In [ ]:
# table 3 output
train_df = pd.read_excel('../data/Intermediate/train.xlsx'); valid_df = pd.read_excel('../data/Intermediate/valid.xlsx'); test_df = pd.read_excel('../data/Intermediate/test.xlsx')


def get_unique_ent_rel(dataframe):
    arg1 = [eval(item)[0] for item in dataframe['triple'].to_list()]
    arg2 = [eval(item)[2] for item in dataframe['triple'].to_list()]
    rel = [eval(item)[1] for item in dataframe['triple'].to_list()]
    print(f'Number of Questions\t:\t{len(dataframe)}')
    print(f'Entity 1\t:\t{len(set(arg1))}')
    print(f'Entity 2\t:\t{len(set(arg2))}')
    print(f'Relations\t:\t{len(set(rel))}')
    print(f'Total Unique Entities\t:\t{len(set(arg1+arg2))}')
    tokenizer = lambda string:string.strip().lower().split()
    tokenized_questions = dataframe['Question'].astype(str).apply(tokenizer).to_list()
    flatten_tokenized_questions = [item for sublist in tokenized_questions for item in sublist]
    print(f'Unique Words\t:\t{len(set(flatten_tokenized_questions))}')

print("*** Training ***")
get_unique_ent_rel(train_df)
print("\n*** Validation ***")
get_unique_ent_rel(valid_df)
print("\n*** Test ***")
get_unique_ent_rel(test_df)


*** Training ***
Number of Questions	:	9921
Entity 1	:	6549
Entity 2	:	5971
Relations	:	4568
Total Unique Entities	:	12155
Unique Words	:	10123

*** Validation ***
Number of Questions	:	1751
Entity 1	:	1624
Entity 2	:	1566
Relations	:	1297
Total Unique Entities	:	3147
Unique Words	:	2958

*** Test ***
Number of Questions	:	5003
Entity 1	:	4064
Entity 2	:	3827
Relations	:	2939
Total Unique Entities	:	7702
Unique Words	:	6314


In [ ]:
# table 5 output {GRU's row}
!python ./BuboQA/entities/train.py  --entity_detection_mode GRU \
                                    --fix_embed --data_dir ../data/SimpleQuestionNotationEntity \
                                    --batch_size 256 \
                                    --vector_cache ../data/Cache/sq_glove300d.pt \
                        


Note: You are using GPU for training
Embedding match number 8546 out of 13972
Shift model to GPU
Namespace(entity_detection_mode='GRU', cuda=True, gpu=0, epochs=30, batch_size=256, dataset='EntityDetection', lr=0.0001, seed=3435, dev_every=2000, log_every=1000, patience=10, save_path='saved_checkpoints', specify_prefix='id1', words_dim=300, num_layer=2, rnn_fc_dropout=0.3, input_size=300, hidden_size=300, rnn_dropout=0.3, clip_gradient=0.6, vector_cache='../data/Cache/sq_glove300d.pt', weight_decay=0, train_embed=False, hits=100, trained_model='', data_dir='../data/SimpleQuestionNotationEntity', results_path='query_text', words_num=13972, label=4)
VOCAB num 13972
Train instance 9921
Dev instance 1751
Test instance 5003
Entity Type 4
EntityDetection(
  (embed): Embedding(13972, 300)
  (gru): GRU(300, 300, num_layers=2, dropout=0.3, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (relu): ReLU()
  (hidden2tag): Sequential(
    (0): Linear(in_features=600, out_features=600

In [ ]:
# table 5 output {LSTM's row}
!python ./BuboQA/entities/train.py  --entity_detection_mode LSTM \
                                    --fix_embed --data_dir ../data/SimpleQuestionNotationEntity \
                                    --batch_size 256 \
                                    --vector_cache ../data/Cache/sq_glove300d.pt \
                        


Note: You are using GPU for training
Embedding match number 8546 out of 13972
Shift model to GPU
Namespace(entity_detection_mode='LSTM', cuda=True, gpu=0, epochs=30, batch_size=256, dataset='EntityDetection', lr=0.0001, seed=3435, dev_every=2000, log_every=1000, patience=10, save_path='saved_checkpoints', specify_prefix='id1', words_dim=300, num_layer=2, rnn_fc_dropout=0.3, input_size=300, hidden_size=300, rnn_dropout=0.3, clip_gradient=0.6, vector_cache='../data/Cache/sq_glove300d.pt', weight_decay=0, train_embed=False, hits=100, trained_model='', data_dir='../data/SimpleQuestionNotationEntity', results_path='query_text', words_num=13972, label=4)
VOCAB num 13972
Train instance 9921
Dev instance 1751
Test instance 5003
Entity Type 4
EntityDetection(
  (embed): Embedding(13972, 300)
  (lstm): LSTM(300, 300, num_layers=2, dropout=0.3, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (relu): ReLU()
  (hidden2tag): Sequential(
    (0): Linear(in_features=600, out_features=

In [ ]:
# table 6 output {LSTM's row}
!python ./BuboQA/relations/train.py  --relation_prediction_mode LSTM \
                                     --fix_embed --data_dir ../data/SimpleQuestionNotationRelation \
                                     --batch_size 256 \
                                     --vector_cache ../data/Cache/sq_glove300d.pt 

Note: You are using GPU for training
Embedding match number 8546 out of 13972
Shift model to GPU
Namespace(relation_prediction_mode='LSTM', cuda=True, gpu=0, epochs=30, batch_size=256, dataset='RelationPrediction', mode='static', lr=0.0001, seed=3435, dev_every=2000, log_every=1000, patience=10, save_path='saved_checkpoints', specify_prefix='id1', output_channel=300, words_dim=300, num_layer=2, rnn_dropout=0.3, input_size=300, hidden_size=300, rnn_fc_dropout=0.3, clip_gradient=0.6, vector_cache='../data/Cache/sq_glove300d.pt', weight_decay=0, cnn_dropout=0.5, train_embed=False, hits=5, data_dir='../data/SimpleQuestionNotationRelation', trained_model='', results_path='results', words_num=13972, rel_label=4236)
VOCAB num 13972
Train instance 9922
Dev instance 1752
Test instance 5004
Relation Type 4236
RelationPrediction(
  (embed): Embedding(13972, 300)
  (lstm): LSTM(300, 300, num_layers=2, dropout=0.3, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (relu): ReLU()
  (h

In [ ]:
# table 6 output {CNN's row}
!python ./BuboQA/relations/train.py  --relation_prediction_mode CNN \
                                     --fix_embed --data_dir ../data/SimpleQuestionNotationRelation \
                                     --batch_size 256 \
                                     --vector_cache ../data/Cache/sq_glove300d.pt 

Note: You are using GPU for training
Embedding match number 8546 out of 13972
Shift model to GPU
Namespace(relation_prediction_mode='CNN', cuda=True, gpu=0, epochs=30, batch_size=256, dataset='RelationPrediction', mode='static', lr=0.0001, seed=3435, dev_every=2000, log_every=1000, patience=10, save_path='saved_checkpoints', specify_prefix='id1', output_channel=300, words_dim=300, num_layer=2, rnn_dropout=0.3, input_size=300, hidden_size=300, rnn_fc_dropout=0.3, clip_gradient=0.6, vector_cache='../data/Cache/sq_glove300d.pt', weight_decay=0, cnn_dropout=0.5, train_embed=False, hits=5, data_dir='../data/SimpleQuestionNotationRelation', trained_model='', results_path='results', words_num=13972, rel_label=4236)
VOCAB num 13972
Train instance 9922
Dev instance 1752
Test instance 5004
Relation Type 4236
RelationPrediction(
  (embed): Embedding(13972, 300)
  (conv1): Conv2d(1, 300, kernel_size=(2, 300), stride=(1, 1), padding=(1, 0))
  (conv2): Conv2d(1, 300, kernel_size=(3, 300), stride=(1,

In [ ]:
!pwd

/content/CodeOceanBERTQA/code/CodeOceanBERTQA/code/src


In [ ]:
# table 7 & 8 & 9 & 10 output


%cd /content/CodeOceanBERTQA/code/src
# model types : [MultiDepthNodeEdgeDetector, BertLSTMCRF, BertCNN, NodeEdgeDetector]
# cross validation: [True, False]
!python train.py NodeEdgeDetector sq valid




/content/CodeOceanBERTQA/code/src
2023-03-17 13:18:53.257056: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 13:18:54.619821: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-17 13:18:54.619963: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023

In [12]:
%cd /content/CodeOceanBERTQA/code/src
# model types : [MultiDepthNodeEdgeDetector, BertLSTMCRF, BertCNN, NodeEdgeDetector]
# cross validation: [True, False]
!python train.py True NodeEdgeDetector rsq test



/content/CodeOceanBERTQA/code/src
2023-03-19 17:55:57.772869: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-19 17:55:58.686035: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-19 17:55:58.686160: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-19 17:55:58.686184: W tensorflow/compiler/tf

In [24]:
%cd /content/CodeOceanBERTQA/code/src
!python evaluation.py tfidf

/content/CodeOceanBERTQA/code/src
2023-03-19 19:02:47.894730: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-19 19:02:48.866128: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-19 19:02:48.866240: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-19 19:02:48.866258: W tensorflow/compiler/tf

In [8]:
# table 14
from src.utils import get_hit
import pandas as pd
test_df = pd.read_excel('../data/Intermediate/test.xlsx')
actual = test_df['Reverb_no'].to_list()
system_results = pd.read_excel('../data/Candidates/test_results.xlsx')['sys'].apply(lambda item:eval(item))
print(get_hit(actual, system_results))
test_df = pd.read_excel('../data/Intermediate/valid.xlsx')
actual = test_df['Reverb_no'].to_list()
system_results = pd.read_excel('../data/Candidates/valid_results.xlsx')['sys'].apply(lambda item:eval(item))
print(get_hit(actual, system_results))

(0.732560463721767, 0.8474915050969418, 0.8648810713571857, 0.878273036178293, 0.8924645212872276)
(0.7572815533980582, 0.8669331810394061, 0.8800685322672759, 0.8897772701313535, 0.903483723586522)


In [1]:
import pandas as pd
from collections import Counter
test = pd.read_excel('../data/Candidates/test_results.xlsx', engine ='openpyxl')
valid = pd.read_excel('../data/Candidates/valid_results.xlsx', engine ='openpyxl')
rr = lambda row: 1/10000 if int(row['Reverb_no']) not in [item[0] for item in eval(row['sys'])] else 1/(1+[item[0] for item in eval(row['sys'])].index(row['Reverb_no']))
test['RR'] = test.apply(rr, axis=1)
valid['RR'] = valid.apply(rr, axis=1)
print(test['RR'].mean(), valid['RR'].mean())

0.7810465810353977 0.801262243574989


In [61]:
from thefuzz import fuzz
import pandas as pd
refrence = pd.read_excel('../data/Intermediate/test.xlsx')
refrence.Question = refrence.Question.apply(lambda x:str(x).lower().strip())
error_dataframe = {
                    'Question':[],
                    'candidates':[],
                    'actual':[],
                    'node':[],
                    'edge':[]
                }
with open('../data/Detailed/Test_Set_With.txt', 'r') as res:
    for line in res:
        if line.find('Question')!=-1:
            temp = eval(line.split(': ')[1].strip())
            error_dataframe['Question'].append(' '.join(temp))
        elif line.find('Sorted candidates')!=-1:
            error_dataframe['candidates'].append(eval(line.split(': ')[1].strip()))
        elif line.find('Node: ')!=-1:
            line = line.replace(', Edge','').split(': ')
            error_dataframe['node'].append(line[1])
            error_dataframe['edge'].append(line[2].strip())
        elif line.find('Actual line number')!=-1:
            error_dataframe['actual'].append(eval(line.split(': ')[1].strip()))
error_dataframe['Meaningful'] = [1 for _ in error_dataframe['actual']]
# for k,v in error_dataframe.items():
#     print(k, len(v), v[:5])
#     error_dataframe[k]=v[:5350]
error_df = pd.DataFrame(error_dataframe)       

In [64]:
error_analysis = pd.merge(error_df, refrence, how='inner', on='Question')
error_analysis['candidates_index'] = error_analysis['candidates'].apply(lambda item:[i[0] for i in item])
gh = lambda row:row['candidates_index'].index(row['actual']) if (row['actual'] in row['candidates_index']) else 100
error_analysis['hit'] = error_analysis.apply(gh, axis=1)
empty_condidates = error_analysis[(error_analysis['hit']<10)&(error_analysis['hit']>-1)][['Question', 'node', 'edge', 'normalized_triple', 'Reverb_no', 'hit']]
empty_condidates['triple'] = empty_condidates.normalized_triple.apply(lambda x:list(str(item).lower() for item in eval(x)))

In [65]:
node_precision, edge_precision = [], []
print(len(empty_condidates))
for index, row in empty_condidates.iterrows():
    temp = max([fuzz.ratio(item, row['node']) for item in row['triple']])
    node_precision.append(temp) 
    temp = max([fuzz.ratio(item, row['edge']) for item in row['triple']])
    edge_precision.append(temp) 
# print(min(node_precision))
print(sum(node_precision)/len(node_precision))
print(sum(edge_precision)/len(edge_precision))

3041
98.5386386057218
77.67181848076291


In [10]:
98.4924990852543
78.15331137943652

98.52592341579125
77.76821416468994


98.54240106418357
77.73761223811107

98.5386386057218
77.66425517921736

98.54151624548736
77.65408598621595




99.40541529454812
92.34687156970362
